# OFA³: Automatic Selection of the Best Non-dominated Sub-networks for Ensembles

- **Description**: 
  - This Jupyter notebook runs the OFA³ selection method for obtaining efficient ensembles. 
  - The search is guided by an evolutionary multi-objective optimization algorithm (EMOA). Here we compare three EMOA:
    - NSGA-II
    - SMS-EMOA
    - SPEA2
  - The objective functions to be optimized are the ImageNet top-1 % accuracy and latency.
  - In these experiments we consider the latency of the ensemble equals to the **<ins>maximum latency</ins>** among the neural networks in the ensemble.

- **Input**: The input of the OFA³ selection method is a set of neural networks.

- **Output**: The output of the OFA³ selection method is a set of efficient ensembles. The number of the components of the ensemble and the components themselves (i.e., the neural networks that compose the ensemble) are automatically found by the algorithm. 

- **arXiv link**: TBA

- **Author**: TBA (hidden due to blind review)
- **email**: TBA (hidden due to blind review)

# Imports

In [1]:
# general
import os
import time
import math
import random
import pickle
from tqdm.notebook import tqdm

# AI/ML/NN
import torch
import numpy as np
import pandas as pd
from torchvision import datasets, transforms

# pymoo
from pymoo.core.individual import Individual
from pymoo.core.population import Population
from pymoo.core.variable import Choice
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.sampling import Sampling
from pymoo.core.callback import Callback
from pymoo.operators.mutation.rm import ChoiceRandomMutation
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.termination import get_termination
from pymoo.util.display.output import Output
from pymoo.util.display.column import Column
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.sms import SMSEMOA
from pymoo.algorithms.moo.spea2 import SPEA2
from pymoo.optimize import minimize
#from pymoo.indicators.hv import HV

In [2]:
# set random seeds for reproducibility
random_seed = 1
random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)
#----------------------------
seed1 = 1
seed2 = 2
seed3 = 3

In [3]:
# set device to use GPU or CPU
cuda_available = torch.cuda.is_available()
if cuda_available:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.cuda.manual_seed(random_seed)
    cuda0 = torch.device('cuda:0')
    print("Using GPU.")
else:
    print("Using CPU.")

Using GPU.


In [4]:
# colors used in matplotlib curves
blue   = '#1f77b4'
orange = '#ff7f0e'
green  = '#2ca02c'
red    = '#d62728'
purple = '#9467bd'
brown  = '#8c564b'
pink   = '#e377c2'
gray   = '#7f7f7f'
olive  = '#bcbd22'
cyan   = '#17becf'
black  = '#000000'

# Get OFA network

In [5]:
#ofa_network = ofa_net("ofa_mbv3_d234_e346_k357_w1.2", pretrained=True)
# ofa_network2 = torch.load(model_dir='~/model/ofa_mbv3_d234_e346_k357_w1.2')

# Dataset & DataLoader

In [6]:
# ImageNet Full
imagenet_data_path = "~/dataset/imagenet/"
#----------------------------
# ImageNet subset
#imagenet_data_path = "~/dataset/imagenet_1k"

In [7]:
# os.makedirs(imagenet_data_path, exist_ok=True)
# download_url('https://hanlab.mit.edu/files/OnceForAll/ofa_cvpr_tutorial/imagenet_1k.zip', model_dir='data')
#! cd data && unzip imagenet_1k 1>/dev/null && cd ..
#! cp -r data/imagenet_1k/* $imagenet_data_path
#! rm -rf data

In [8]:
# The following function build the data transforms for test
def build_val_transform(size):
    return transforms.Compose(
        [
            transforms.Resize(int(math.ceil(size / 0.875))),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

In [9]:
#imagenet_train = datasets.ImageFolder(root=os.path.join(imagenet_data_path, "train"), transform=build_val_transform(224))
imagenet_train_subset = datasets.ImageFolder(root=os.path.join(imagenet_data_path, "train_subset_50k"), transform=build_val_transform(224))
imagenet_val = datasets.ImageFolder(root=os.path.join(imagenet_data_path, "val"), transform=build_val_transform(224))

In [10]:
## this dataloader is for the training set --> used to guide the search
#data_loader_train = torch.utils.data.DataLoader(
#    imagenet_train,
#    batch_size=256,   # evaluation batch size
#    shuffle=False,    # evaluation only
#    num_workers=4,    # number of workers for the data loader
#    pin_memory=True,
#    drop_last=False,
#)
#print("The ImageNet dataloader for the training set is ready.")

In [11]:
# this dataloader is for the 50k subset of the training set --> used to generate probability table
data_loader_train_subset = torch.utils.data.DataLoader(
    datasets.ImageFolder(root=os.path.join(imagenet_data_path, "train_subset_50k"), transform=build_val_transform(224)),
    batch_size=1_024, # evaluation batch size
    shuffle=False,    # evaluation only
    num_workers=4,    # number of workers for the data loader
    pin_memory=True,
    drop_last=False,
)
print("The ImageNet dataloader for the training set is ready.")

The ImageNet dataloader for the training set is ready.


In [12]:
# this dataloader is for the validation set --> used to measure performance
#data_loader_val = torch.utils.data.DataLoader(
#    imagenet_val,
#    batch_size=1_024, # test batch size
#    shuffle=False,    # evaluation only
#    num_workers=4,    # number of workers for the data loader
#    pin_memory=True,
#    drop_last=False,
#)
#print("The ImageNet dataloader for the validation set is ready.")

# Load the probability & class outputs of the OFA² models

In [13]:
# load last population of OFA² search
with open('ofa2_nsga2.pickle', 'rb') as f:
    ofa2_nsga2 = pickle.load(f)
#----------------------------
# get latency of the models
ofa2_nsga2_latency = ofa2_nsga2.get('F')[:,0]

These tables are obtained evaluating the architectures found by the OFA² search

In [14]:
n_outputs = 5                                   # number of outputs considered to build the cumulative table
n_models = len(ofa2_nsga2)                      # OFA² last population size: 100
ds_size = len(imagenet_train_subset)            # ImageNet training set size: 50,000
n_classes = len(imagenet_train_subset.classes)  # ImageNet classes: 1,000 
#----------------------------
#ds_size = len(imagenet_train_subset)            # ImageNet training set size: 1,281,167
#n_classes = len(imagenet_train_subset.classes)  # ImageNet classes: 1,000 

In [15]:
# create class and probabilities tables
top5_class = np.empty((n_models, ds_size, n_outputs), dtype=np.int16)   # shape = (100 x 1281167 x 5)
top5_prob = np.empty((n_models, ds_size, n_outputs), dtype=np.float32)  # shape = (100 x 1281167 x 5)
#-------------------------------------------------
# fill table with data from csv files of the 100 models found by OFA² search
for i in tqdm(range(n_models)):
    top5_class[i] = pd.read_csv('ofa2_models_output_subset_50k/OFA2_model_' + str(i).zfill(3) + '_class.csv', usecols=[0,1,2,3,4], header=None).values
    top5_prob[i] = pd.read_csv('ofa2_models_output_subset_50k/OFA2_model_' + str(i).zfill(3) + '_prob.csv', usecols=[0,1,2,3,4], header=None).values
#-------------------------------------------------
# ground truth labels
labels = pd.read_csv('ofa2_models_output_subset_50k/OFA2_model_' + str(0).zfill(3) + '_class.csv', usecols=[5], header=None).values.squeeze()

  0%|          | 0/100 [00:00<?, ?it/s]

# Fill the cumulative probability lookup table

In [16]:
# create cumulative lookup table of probabilities: shape = (100 x 1281167 x 1000)
cumulative_lut = np.empty((n_models, ds_size, n_classes), dtype=np.float32)

In [17]:
# start measuring time
start_time = time.time()
#-------------------------------------------------
# loop over models (0~99)
for model_id in tqdm(range(n_models), position=1):
    # loop over each image output class (0~1281166): get lines of top5_class table
    #for i, output_classes in enumerate(tqdm(top5_class[model_id], position=0, leave=False)):
    for i, output_classes in enumerate(top5_class[model_id]):
        # loop over each top-5 outputs (0~4): get rows of top5_class table & respective rows of top5_prob
        for j, topk in enumerate(output_classes):
            cumulative_lut[model_id, i, topk] += top5_prob[model_id, i, j]
#-------------------------------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The creation of the cumulative probability lookup tables took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

  0%|          | 0/100 [00:00<?, ?it/s]

The creation of the cumulative probability lookup tables took 00h00m25s to finish.


# Hyperparameters

In [18]:
""" Hyperparameters
- P: size of the population in each generation (number of individuals)
- N: number of generations to run the algorithm
- mutate_prob: probability of gene mutation in the evolutionary search
"""
P = 100
N = 2_000
mutation_prob = 0.01

# Sum Latency (components: n > 2)

## Individual & Population

In [19]:
class EnsembleIndividual(Individual):
    def __init__(self, ensemble_selection, config=None, **kwargs):
        super().__init__(config=None, **kwargs)
        self.X = ensemble_selection

In [20]:
# individuals with all genes equals to "1"
pop = Population(individuals=[EnsembleIndividual(ind) for ind in np.ones((P,P))])

## Search Space & Problem

In [21]:
# participation
part_var = Choice(options=[0, 1])
#----------------------------
# search space
ensemble_size = P

In [22]:
class EnsembleProblemSoft(ElementwiseProblem):
    def __init__(self, ensemble_size, cumulative_table, latency_table, label, device, metric='max'):
        super().__init__(
            # n_var = 100,
            vars=ensemble_size * [part_var],
            n_obj=2,
            n_constr=1,
        )
        self.device = device  # allocate in GPU 
        self.label = torch.tensor(label).to(self.device)
        self.val_size = torch.tensor(len(label)).to(self.device)
        self.N_CLASSES = 1_000
        self.latency_table = latency_table
        self.metric = metric
        # reshape to (50k x 1k x 100)
        self.cumulative_table = torch.permute(torch.tensor(cumulative_table), (1, 2, 0)).to(self.device)
        
        
    def calc_ensemble_lat_sum(self, x):
        return np.inner(x, self.latency_table)
    
    
    def calc_ensemble_lat_max(self, x):
        return max(np.multiply(x, self.latency_table))
    
    
    def calc_ensemble_accuracy(self, x):
        ind = torch.tensor(x, device=self.device, dtype=torch.float32)
        out = torch.matmul(self.cumulative_table, ind)
        acc = torch.count_nonzero(torch.argmax(out, axis=1) == self.label) / self.val_size * 100
        return acc.item()
    
    
    def _evaluate(self, x, out, *args, **kwargs):
        if self.metric == 'sum':
            f1 = self.calc_ensemble_lat_sum(x)   # sum latency
        else:
            f1 = self.calc_ensemble_lat_max(x)   # max latency
        acc = self.calc_ensemble_accuracy(x)
        f2 = 100 - acc
        g1 = 2 - np.count_nonzero(x)
        out["F"] = np.column_stack([f1, f2])
        out["G"] = [g1]

## Operators

In [23]:
# Mutation: Random
ensemble_mutation_rc = ChoiceRandomMutation(prob=1.0, prob_var=mutation_prob)

In [24]:
# Crossover (Recombination): Uniform
ensemble_crossover_ux = UniformCrossover(prob=1.0)
parents = [[pop[0], pop[1]] for _ in range(1)]

In [25]:
# Sampling: Random
class EnsembleSampling(Sampling):
    def _do(self, ensemble_problem, n_samples, **kwargs):
        return [
            [np.random.choice(var.options) for var in ensemble_problem.vars]
            for _ in range(n_samples)
        ]
#----------------------------
ensemble_sampling = EnsembleSampling()

## Termination

In [26]:
# convergence
#termination_default = DefaultMultiObjectiveTermination(xtol=1e-8, cvtol=1e-6, ftol=0.0025, period=30, n_max_gen=1000, n_max_evals=100000)
#----------------------------
# generations
termination_gen = get_termination("n_gen", N)

## Algorithm & Optimization

In [27]:
class MyOutput(Output):

    def __init__(self):
        super().__init__()
        self.n_gen = Column("n_gen", width=5)
        #self.g_mean = Column("g_mean", width=6)
        #self.g_min = Column("g_mean", width=6)
        #self.g_max = Column("g_mean", width=6)
        self.lat_min = Column("lat_min", width=15)
        self.lat_max = Column("lat_max", width=15)
        self.acc_min = Column("acc_min", width=15)
        self.acc_max = Column("acc_max", width=15)
        self.acc_mean = Column("acc_mean", width=15)
        #self.columns = [self.n_gen, self.g_mean, self.lat_min, self.lat_max, self.acc_min, self.acc_max, self.acc_mean]
        #self.columns = [self.n_gen, self.g_min, self.g_max, self.lat_min, self.lat_max, self.acc_min, self.acc_max, self.acc_mean]
        self.columns = [self.n_gen, self.lat_min, self.lat_max, self.acc_min, self.acc_max, self.acc_mean]
        #self.columns = [self.n_gen, self.g_mean]#, self.lat_min, self.lat_max, self.acc_min, self.acc_max, self.acc_mean]
        #self.columns = [self.n_gen, self.lat_min, self.lat_max, self.acc_min, self.acc_max, self.acc_mean]

    def update(self, algorithm):
        super().update(algorithm)
        self.n_gen.set(algorithm.n_gen)
        #self.g_mean.set(2 - algorithm.pop.get("G"))
        #self.g_mean.set(np.mean(2 - algorithm.pop.get("G")[:]).item())
        #self.g_min.set(np.abs(np.min(algorithm.pop.get("G"))))
        #self.g_max.set(np.abs(np.max(algorithm.pop.get("G"))))
        self.lat_min.set(np.min(algorithm.pop.get("F")[:,0]))
        self.lat_max.set(np.max(algorithm.pop.get("F")[:,0]))
        self.acc_min.set(np.min(100 - algorithm.pop.get("F")[:,1]))
        self.acc_max.set(np.max(100 - algorithm.pop.get("F")[:,1]))
        self.acc_mean.set(np.mean(100 - algorithm.pop.get("F")[:,1]))

### Max Latency

In [28]:
ensemble_problem_max = EnsembleProblemSoft(P, cumulative_lut, ofa2_nsga2_latency, labels, device=cuda0, metric='max')

#### NSGA-II

In [29]:
class Nsga2Callback(Callback):
    
    def __init__(self, metric='max', seed=1) -> None:
        super().__init__()
        self.data = []
        self.metric = metric
        self.seed = seed
        
    def notify(self, algorithm):
        pop_data = algorithm.n_gen, algorithm.pop.get("X"), algorithm.pop.get("F"), algorithm.pop.get("G")
        filename = 'gen_' + str(algorithm.n_gen).zfill(4) + '.pickle'
        with open(os.path.join('ofa3_ensemble', 'NSGA-II', 'soft-lat' + self.metric + '-generation', 'seed_' + str(self.seed), filename), 'bw') as f:
            pickle.dump(pop_data, f)

In [30]:
ensemble_algorithm_nsga2 = NSGA2(
    pop_size=P,
    sampling=Population(individuals=[EnsembleIndividual(ind) for ind in np.ones((P,P))]),
    #sampling=Population(individuals=[EnsembleIndividual(ind) for ind in np.identity(P)]),
    crossover=ensemble_crossover_ux,
    mutation=ensemble_mutation_rc,
)

##### seed 1

In [31]:
torch.manual_seed(seed1)
np.random.seed(seed1)
random.seed(seed1)

In [32]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_nsga2_max_seed1 = minimize(
    ensemble_problem_max,
    ensemble_algorithm_nsga2,
    callback=Nsga2Callback(metric='max', seed=seed1),
    termination=termination_gen,
    seed=seed1,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  4.62159434E+01 |  4.62159434E+01 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  4.62159434E+01 |  4.62159434E+01 |  9.09919968E+01 |  9.10440063E+01 |  9.10177000E+01
    3 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10559998E+01 |  9.10296804E+01
    4 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10640030E+01 |  9.10360200E+01
    5 |  4.51405232E+01 |  4.62159434E+01 |  9.09940033E+01 |  9.10640030E+01 |  9.10411203E+01
    6 |  4.51405232E+01 |  4.62159434E+01 |  9.10059967E+01 |  9.10699997E+01 |  9.10468198E+01
    7 |  4.51405232E+01 |  4.62159434E+01 |  9.10179977E+01 |  9.10740051E+01 |  9.10537199E+01
    8 |  4.51405232E+01 |  4.62159434E+01 |  9.10299988E+01 |  9.10880051E+01 |  9.10591197E+01
    9 |  4.51405232E+01 |  4.62159434E+01 |  9.10480042E+01 |  9.10920029E+01 |  9.10673611E+01
   10 |  4.50088106E+01 |  4.62159434E+0

In [33]:
del ensemble_nsga2_max_seed1
torch.cuda.empty_cache()

##### seed 2

In [34]:
torch.manual_seed(seed2)
np.random.seed(seed2)
random.seed(seed2)

In [35]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_nsga2_max_seed2 = minimize(
    ensemble_problem_max,
    ensemble_algorithm_nsga2,
    callback=Nsga2Callback(metric='max', seed=seed2),
    #termination=termination_gen,
    termination=get_termination("n_gen", 1_500),
    seed=seed2,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  4.62159434E+01 |  4.62159434E+01 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  4.62159434E+01 |  4.62159434E+01 |  9.09959946E+01 |  9.10380020E+01 |  9.10176598E+01
    3 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10499954E+01 |  9.10285408E+01
    4 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10599976E+01 |  9.10360001E+01
    5 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10680008E+01 |  9.10426010E+01
    6 |  4.51405232E+01 |  4.62159434E+01 |  9.10059967E+01 |  9.10699997E+01 |  9.10464597E+01
    7 |  4.51405232E+01 |  4.62159434E+01 |  9.10240021E+01 |  9.10820007E+01 |  9.10543599E+01
    8 |  4.51405232E+01 |  4.62159434E+01 |  9.10400009E+01 |  9.10839996E+01 |  9.10621195E+01
    9 |  4.51405232E+01 |  4.62159434E+01 |  9.10540009E+01 |  9.10920029E+01 |  9.10682610E+01
   10 |  4.51405232E+01 |  4.62159434E+0

In [36]:
del ensemble_nsga2_max_seed2
torch.cuda.empty_cache()

##### seed 3

In [37]:
torch.manual_seed(seed3)
np.random.seed(seed3)
random.seed(seed3)

In [38]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_nsga2_max_seed3 = minimize(
    ensemble_problem_max,
    ensemble_algorithm_nsga2,
    callback=Nsga2Callback(metric='max', seed=seed3),
    termination=termination_gen,
    seed=seed3,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  4.62159434E+01 |  4.62159434E+01 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  4.51405232E+01 |  4.62159434E+01 |  9.09919968E+01 |  9.10419998E+01 |  9.10164994E+01
    3 |  4.51405232E+01 |  4.62159434E+01 |  9.09959946E+01 |  9.10480042E+01 |  9.10263408E+01
    4 |  4.51405232E+01 |  4.62159434E+01 |  9.09799957E+01 |  9.10699997E+01 |  9.10306406E+01
    5 |  4.50088106E+01 |  4.62159434E+01 |  9.09840012E+01 |  9.10699997E+01 |  9.10344405E+01
    6 |  4.50088106E+01 |  4.62159434E+01 |  9.09840012E+01 |  9.10699997E+01 |  9.10460203E+01
    7 |  4.50088106E+01 |  4.62159434E+01 |  9.09840012E+01 |  9.10780029E+01 |  9.10535399E+01
    8 |  4.50088106E+01 |  4.62159434E+01 |  9.09840012E+01 |  9.10800018E+01 |  9.10551797E+01
    9 |  4.50088106E+01 |  4.62159434E+01 |  9.10179977E+01 |  9.10820007E+01 |  9.10617805E+01
   10 |  4.50088106E+01 |  4.62159434E+0

In [39]:
del ensemble_nsga2_max_seed3
torch.cuda.empty_cache()

#### SMS-EMOA

In [40]:
class SmsemoaCallback(Callback):
    
    def __init__(self, metric='max', seed=1) -> None:
        super().__init__()
        self.data = []
        self.metric = metric
        self.seed = seed
        
    def notify(self, algorithm):
        pop_data = algorithm.n_gen, algorithm.pop.get("X"), algorithm.pop.get("F"), algorithm.pop.get("G")
        filename = 'gen_' + str(algorithm.n_gen).zfill(4) + '.pickle'
        with open(os.path.join('ofa3_ensemble', 'SMS-EMOA', 'soft-lat' + self.metric + '-generation', 'seed_' + str(self.seed), filename), 'bw') as f:
            pickle.dump(pop_data, f)

In [41]:
ensemble_algorithm_smsemoa = SMSEMOA(
    pop_size=P,
    # sampling=ensemble_sampling,
    # sampling=Population(individuals=[EnsembleIndividual(ind) for ind in np.identity(P)]),
    sampling=Population(individuals=[EnsembleIndividual(ind) for ind in np.ones((P,P))]),
    crossover=ensemble_crossover_ux,
    mutation=ensemble_mutation_rc,
)

##### seed 1

In [42]:
torch.manual_seed(seed1)
np.random.seed(seed1)
random.seed(seed1)

In [43]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_smsemoa_max_seed1 = minimize(
    ensemble_problem_max,
    ensemble_algorithm_smsemoa,
    callback=SmsemoaCallback(metric='max', seed=seed1),
    termination=termination_gen,
    seed=seed1,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  4.62159434E+01 |  4.62159434E+01 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  4.62159434E+01 |  4.62159434E+01 |  9.09919968E+01 |  9.10440063E+01 |  9.10177000E+01
    3 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10559998E+01 |  9.10296804E+01
    4 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10599976E+01 |  9.10376198E+01
    5 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10780029E+01 |  9.10457210E+01
    6 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10880051E+01 |  9.10527798E+01
    7 |  4.51405232E+01 |  4.62159434E+01 |  9.10120010E+01 |  9.10880051E+01 |  9.10590999E+01
    8 |  4.51405232E+01 |  4.62159434E+01 |  9.10260010E+01 |  9.10899963E+01 |  9.10643008E+01
    9 |  4.51405232E+01 |  4.62159434E+01 |  9.10480042E+01 |  9.10920029E+01 |  9.10727606E+01
   10 |  4.51405232E+01 |  4.62159434E+0

In [44]:
del ensemble_smsemoa_max_seed1
torch.cuda.empty_cache()

##### seed 2

In [45]:
torch.manual_seed(seed2)
np.random.seed(seed2)
random.seed(seed2)

In [46]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_smsemoa_max_seed2 = minimize(
    ensemble_problem_max,
    ensemble_algorithm_smsemoa,
    callback=SmsemoaCallback(metric='max', seed=seed2),
    termination=termination_gen,
    seed=seed2,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  4.62159434E+01 |  4.62159434E+01 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  4.62159434E+01 |  4.62159434E+01 |  9.09959946E+01 |  9.10380020E+01 |  9.10176598E+01
    3 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10499954E+01 |  9.10285408E+01
    4 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10599976E+01 |  9.10360598E+01
    5 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10699997E+01 |  9.10431200E+01
    6 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10839996E+01 |  9.10490600E+01
    7 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10880051E+01 |  9.10522197E+01
    8 |  4.51405232E+01 |  4.62159434E+01 |  9.10199966E+01 |  9.10940018E+01 |  9.10573802E+01
    9 |  4.50088106E+01 |  4.62159434E+01 |  9.10120010E+01 |  9.10940018E+01 |  9.10673608E+01
   10 |  4.50088106E+01 |  4.62159434E+0

In [47]:
del ensemble_smsemoa_max_seed2
torch.cuda.empty_cache()

##### seed 3

In [48]:
torch.manual_seed(seed3)
np.random.seed(seed3)
random.seed(seed3)

In [49]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_smsemoa_max_seed3 = minimize(
    ensemble_problem_max,
    ensemble_algorithm_smsemoa,
    callback=SmsemoaCallback(metric='max', seed=seed3),
    termination=termination_gen,
    seed=seed3,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  4.62159434E+01 |  4.62159434E+01 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  4.51405232E+01 |  4.62159434E+01 |  9.09919968E+01 |  9.10419998E+01 |  9.10164994E+01
    3 |  4.51405232E+01 |  4.62159434E+01 |  9.09919968E+01 |  9.10499954E+01 |  9.10278208E+01
    4 |  4.51405232E+01 |  4.62159434E+01 |  9.09840012E+01 |  9.10699997E+01 |  9.10354398E+01
    5 |  4.51405232E+01 |  4.62159434E+01 |  9.09919968E+01 |  9.10699997E+01 |  9.10406999E+01
    6 |  4.51405232E+01 |  4.62159434E+01 |  9.10120010E+01 |  9.10800018E+01 |  9.10494601E+01
    7 |  4.51405232E+01 |  4.62159434E+01 |  9.10319977E+01 |  9.10839996E+01 |  9.10587400E+01
    8 |  4.50088106E+01 |  4.62159434E+01 |  9.10359955E+01 |  9.10899963E+01 |  9.10632208E+01
    9 |  4.50088106E+01 |  4.62159434E+01 |  9.10179977E+01 |  9.11259995E+01 |  9.10680606E+01
   10 |  4.50088106E+01 |  4.62159434E+0

In [50]:
del ensemble_smsemoa_max_seed3
torch.cuda.empty_cache()

#### SPEA2

In [51]:
class Spea2Callback(Callback):
    
    def __init__(self, metric='max', seed=1) -> None:
        super().__init__()
        self.data = []
        self.metric = metric
        self.seed = seed
        
    def notify(self, algorithm):
        pop_data = algorithm.n_gen, algorithm.pop.get("X"), algorithm.pop.get("F"), algorithm.pop.get("G")
        filename = 'gen_' + str(algorithm.n_gen).zfill(4) + '.pickle'
        with open(os.path.join('ofa3_ensemble', 'SPEA2', 'soft-lat' + self.metric + '-generation', 'seed_' + str(self.seed), filename), 'bw') as f:
            pickle.dump(pop_data, f)

In [52]:
ensemble_algorithm_spea2 = SPEA2(
    pop_size=P,
    #sampling=ensemble_sampling,
    #sampling=Population(individuals=[EnsembleIndividual(ind) for ind in np.identity(P)]),
    sampling=Population(individuals=[EnsembleIndividual(ind) for ind in np.ones((P,P))]),
    crossover=ensemble_crossover_ux,
    mutation=ensemble_mutation_rc,
)

##### seed 1

In [53]:
torch.manual_seed(seed1)
np.random.seed(seed1)
random.seed(seed1)

In [54]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_spea2_max_seed1 = minimize(
    ensemble_problem_max,
    ensemble_algorithm_spea2,
    callback=Spea2Callback(metric='max', seed=seed1),
    termination=termination_gen,
    seed=seed1,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  4.62159434E+01 |  4.62159434E+01 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01


/opt/conda/lib/python3.10/site-packages/pymoo/algorithms/moo/spea2.py:62: RuntimeWarning: invalid value encountered in divide
  _F = (F - ideal) / (nadir - ideal)


    2 |  4.62159434E+01 |  4.62159434E+01 |  9.09860001E+01 |  9.10440063E+01 |  9.10175199E+01
    3 |  4.51405232E+01 |  4.62159434E+01 |  9.10000000E+01 |  9.10480042E+01 |  9.10268401E+01
    4 |  4.51405232E+01 |  4.62159434E+01 |  9.09879990E+01 |  9.10640030E+01 |  9.10333801E+01
    5 |  4.51405232E+01 |  4.62159434E+01 |  9.09879990E+01 |  9.10660019E+01 |  9.10377998E+01
    6 |  4.51405232E+01 |  4.62159434E+01 |  9.09879990E+01 |  9.10800018E+01 |  9.10378395E+01
    7 |  4.51405232E+01 |  4.62159434E+01 |  9.10260010E+01 |  9.10920029E+01 |  9.10491599E+01
    8 |  4.51405232E+01 |  4.62159434E+01 |  9.10340042E+01 |  9.10999985E+01 |  9.10588406E+01
    9 |  4.50088106E+01 |  4.62159434E+01 |  9.10220032E+01 |  9.10999985E+01 |  9.10673006E+01
   10 |  4.50088106E+01 |  4.62159434E+01 |  9.10220032E+01 |  9.10999985E+01 |  9.10747811E+01
   11 |  4.50088106E+01 |  4.62159434E+01 |  9.10139999E+01 |  9.11019974E+01 |  9.10787413E+01
   12 |  4.50088106E+01 |  4.62159434E+0

In [55]:
del ensemble_spea2_max_seed1
torch.cuda.empty_cache()

##### seed 2

In [56]:
torch.manual_seed(seed2)
np.random.seed(seed2)
random.seed(seed2)

In [57]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_spea2_max_seed2 = minimize(
    ensemble_problem_max,
    ensemble_algorithm_spea2,
    callback=Spea2Callback(metric='max', seed=seed2),
    termination=termination_gen,
    seed=seed2,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  4.62159434E+01 |  4.62159434E+01 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  4.62159434E+01 |  4.62159434E+01 |  9.09940033E+01 |  9.10380020E+01 |  9.10174998E+01
    3 |  4.51405232E+01 |  4.62159434E+01 |  9.10019989E+01 |  9.10520020E+01 |  9.10263203E+01
    4 |  4.50088106E+01 |  4.62159434E+01 |  9.09820023E+01 |  9.10579987E+01 |  9.10332201E+01
    5 |  4.50088106E+01 |  4.62159434E+01 |  9.09820023E+01 |  9.10640030E+01 |  9.10395606E+01
    6 |  4.50088106E+01 |  4.62159434E+01 |  9.09799957E+01 |  9.10719986E+01 |  9.10401010E+01
    7 |  4.50088106E+01 |  4.62159434E+01 |  9.09799957E+01 |  9.10740051E+01 |  9.10370600E+01
    8 |  4.50088106E+01 |  4.62159434E+01 |  9.09799957E+01 |  9.10820007E+01 |  9.10437395E+01
    9 |  4.50088106E+01 |  4.62159434E+01 |  9.10099945E+01 |  9.10839996E+01 |  9.10518795E+01
   10 |  4.50088106E+01 |  4.62159434E+0

In [58]:
del ensemble_spea2_max_seed2
torch.cuda.empty_cache()

##### seed 3

In [59]:
torch.manual_seed(seed3)
np.random.seed(seed3)
random.seed(seed3)

In [60]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_spea2_max_seed3 = minimize(
    ensemble_problem_max,
    ensemble_algorithm_spea2,
    callback=Spea2Callback(metric='max', seed=seed3),
    termination=termination_gen,
    seed=seed3,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  4.62159434E+01 |  4.62159434E+01 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  4.51405232E+01 |  4.62159434E+01 |  9.09919968E+01 |  9.10419998E+01 |  9.10164994E+01
    3 |  4.51405232E+01 |  4.62159434E+01 |  9.09959946E+01 |  9.10520020E+01 |  9.10259606E+01
    4 |  4.51405232E+01 |  4.62159434E+01 |  9.09899979E+01 |  9.10640030E+01 |  9.10292397E+01
    5 |  4.51405232E+01 |  4.62159434E+01 |  9.10080032E+01 |  9.10800018E+01 |  9.10354005E+01
    6 |  4.51405232E+01 |  4.62159434E+01 |  9.10220032E+01 |  9.10800018E+01 |  9.10450409E+01
    7 |  4.50088106E+01 |  4.62159434E+01 |  9.10260010E+01 |  9.10839996E+01 |  9.10543405E+01
    8 |  4.50088106E+01 |  4.62159434E+01 |  9.10220032E+01 |  9.10839996E+01 |  9.10634805E+01
    9 |  4.50088106E+01 |  4.62159434E+01 |  9.10220032E+01 |  9.10940018E+01 |  9.10709202E+01
   10 |  4.50088106E+01 |  4.62159434E+0

In [61]:
del ensemble_spea2_max_seed3
torch.cuda.empty_cache()

# End of Notebook